In [120]:
import pandas as pd
import seaborn as sns
import os
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.misc import electrocardiogram
from scipy import signal
import scipy.integrate as integrate
import scipy.fftpack as ff
#!/usr/bin/env python

import numpy as np, os, sys, joblib
from scipy.io import loadmat
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from get_12ECG_features import get_12ECG_features

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from pathlib import Path
from numpy import genfromtxt

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv1D, LSTM, Dense, Dropout, TimeDistributed, Flatten
from keras.optimizers import Adam
import keras

import matplotlib.pyplot as plt
%matplotlib inline

# 1. Signal lecture - using A0001 sample
#### Please note: 
For easier data analysis and features visualization, please use pandas (since its easier to use). But for the final signal (the one we need to integration), please use numpy format (convert to numpy using "signal.values")

In [154]:
features = f'features.csv'
labels = f'labels.csv'
testing = f'testing.csv'
testing = genfromtxt(testing, delimiter=',')
features = genfromtxt(features, delimiter=',')
labels = genfromtxt(labels, delimiter=',')

In [166]:
testing = tf.keras.utils.normalize(testing)

features = tf.keras.utils.normalize(features)

In [167]:
print(features.dtype)
print(testing.dtype)
print(label.dtype)

float64
float64
float64


In [168]:
print("min testing: ", min(testing), "max testing: ", max(testing))

min testing:  [ 8.18680069e-14  1.07721062e-15  1.35874940e-11  4.27676526e-08
  1.38626234e-11  6.32003060e-08  7.47888383e-12  3.24457787e-08
  5.31321232e-08  1.00000000e+00 -2.32469180e-16 -4.49220193e-16
 -1.20301183e-15 -1.88455344e-15] max testing:  [ 8.18680069e-14  1.07721062e-15  1.35874940e-11  4.27676526e-08
  1.38626234e-11  6.32003060e-08  7.47888383e-12  3.24457787e-08
  5.31321232e-08  1.00000000e+00 -2.32469180e-16 -4.49220193e-16
 -1.20301183e-15 -1.88455344e-15]


In [169]:
testing_reshape = testing.reshape(1, -1)
imputer=SimpleImputer().fit(testing_reshape)
testing_reshape = imputer.transform(testing_reshape)
print(testing_reshape.shape)

(1, 14)


In [170]:
print(features.shape, labels.shape)
4812*14

(4813, 14) (4813, 9)


67368

In [171]:
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(9)
])


In [182]:
print("training the model")
############# MODEL 1 - NN ######################
model_1 = tf.keras.models.Sequential()
model_1.add(tf.keras.layers.Flatten())
# use 128 neurons & use relu act func
model_1.add(tf.keras.layers.Dense(128, activation = tf.nn.relu))
model_1.add(tf.keras.layers.Dense(128, activation = tf.nn.relu))

model_1.add(tf.keras.layers.Dense(128, activation = tf.nn.relu))

model_1.add(tf.keras.layers.Dense(128, activation = tf.nn.relu))

model_1.add(tf.keras.layers.Dense(128, activation = tf.nn.relu))

model_1.add(tf.keras.layers.Dense(128, activation = tf.nn.relu))
# the final output layer must be Dense and must fit # classifications and must use probability distribution instead of an activation function
model_1.add(tf.keras.layers.Dense(9, activation = tf.nn.softmax))
    
# Compile 
model_1.compile(
optimizer= 'adam' ,# NN intends to minimize losss, not maximize accuracy
loss = 'sparse_categorical_crossentropy',
metrics = ['accuracy']
)
label_final = []
    
# Extract hot-encode format to normal format
for i in range(labels.shape[0]):
    label = labels[i]
    decoded_labels = decode(label)
    label_final.append(decoded_labels)
# Convert list to np array
label_final = np.array(label_final)
label_final = np.uint8(label_final)
print(label_final.dtype)
# Fit the model appropiatelly
model_1.fit(features, label_final, epochs=10)
val_loss, val_acc = model_1.evaluate(features, label_final)
print("The model is trained")

training the model
uint8
Epoch 1/3
151/151 [==============================] - 0s 1ms/step - loss: 2.0675 - accuracy: 0.2188
Epoch 2/3
151/151 [==============================] - 0s 2ms/step - loss: 2.0541 - accuracy: 0.2256
Epoch 3/3
151/151 [==============================] - 0s 1ms/step - loss: 2.0534 - accuracy: 0.2256
The model is trained


In [173]:
model_1.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_9 (Flatten)          (None, 14)                0         
_________________________________________________________________
dense_30 (Dense)             (None, 128)               1920      
_________________________________________________________________
dense_31 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_32 (Dense)             (None, 9)                 1161      
Total params: 19,593
Trainable params: 19,593
Non-trainable params: 0
_________________________________________________________________


In [174]:
# Save model.
model_1.save("NN_1.model")

INFO:tensorflow:Assets written to: NN_1.model\assets


In [175]:
print(features.shape, label_final.shape)

(4813, 14) (4813,)


In [176]:
labels

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [177]:
nn_model = tf.keras.models.load_model('NN_1.model/')
nn_model

In [178]:
testing = np.array(testing)
testing.dtype

dtype('float64')

In [179]:
print("min testing: ", min(testing), "max testing: ", max(testing))

min testing:  [ 8.18680069e-14  1.07721062e-15  1.35874940e-11  4.27676526e-08
  1.38626234e-11  6.32003060e-08  7.47888383e-12  3.24457787e-08
  5.31321232e-08  1.00000000e+00 -2.32469180e-16 -4.49220193e-16
 -1.20301183e-15 -1.88455344e-15] max testing:  [ 8.18680069e-14  1.07721062e-15  1.35874940e-11  4.27676526e-08
  1.38626234e-11  6.32003060e-08  7.47888383e-12  3.24457787e-08
  5.31321232e-08  1.00000000e+00 -2.32469180e-16 -4.49220193e-16
 -1.20301183e-15 -1.88455344e-15]


In [180]:
predictions = nn_model.predict([testing_reshape])[0]
predictions

array([0.09430914, 0.19484664, 0.03007795, 0.03726863, 0.10778197,
       0.09311236, 0.12559493, 0.10531376, 0.21169463], dtype=float32)